<a href="https://colab.research.google.com/github/drflopes/FlopesDS/blob/main/Praticas/Pratica_GLabSr02_Pipeline_completo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Prática 2**

**Neste script trabalharemos com um único arq.csv**

Montando um Pipeline com dados da Bolsa de Valores.
Aqui vamos usar a biblioteca yfinance para fazer download dos dados históricos.

-------------------------
Instalando a biblioteca yfinance, não disponível no Colab ou SageMaker

In [4]:
# pip é um comando do SO/Linux e, para rodar no Jupyter/Notebook, é necessário utilizar uma magicfunction, no caso "!"

!pip install pandas_datareader
!pip install yfinance

     |████████████████████████████████| 6.3MB 3.9MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23919 sha256=edbcf5ffbfd708605dc7194936617109de4753535fa3eafab1e9b3e126aae088
  Stored in directory: /root/.cache/pip/wheels/fe/a0/79/b73d4a0c535b421b88fc7b393936b371fabbfeaf979eca4050
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [5]:
import pandas as pd
from pandas_datareader import data as pdr
from datetime import datetime as dt
import yfinance as yf

# Função seguinte sobrescreve os métodos do datareader com os métodos do yfinance
yf.pdr_override()


Criamos um vetor com os Tickers que vamos utilizar - A lista pode ser alterada:

In [6]:
#ticker_list=['DJIA', 'DOW', 'LB', 'EXPE', 'PXD', 'MCHP', 'CRM', 'JEC' , 'NRG', 'HFC', 'NOW']
ticker_list=['SPY', 'DOW', 'LB']

#criando variavel com a data de hoje.
today = dt.today()

print(today)
print(str(today)[0:19])
print(ticker_list)


2021-07-12 18:22:16.258852
2021-07-12 18:22:16
['SPY', 'DOW', 'LB']


Você pode usar a data de hoje ou selecionar um período de coleta:
Também vamos criar uma UDF para salvar arquivos csv, um para cada Ticker


In [7]:
#start_date=dt(2019, 1, 1)
#end_date=dt(2019, 12, 30)
start_date=today
end_date=today

""" 
Vamos criar uma lista (files), para compor os nomes dos arquivos 
considerando tickers cadastrados em ticker_list e data_end
"""

files=[]

def getData(ticker):
  print(ticker)
  data = pdr.get_data_yahoo(ticker, start=start_date, end=end_date)
  data['Ticker'] = ticker
  dataname= ticker+'_'+str(end_date)[0:19]
  print(dataname)
  files.append(dataname)
  SaveData(data, dataname)
  

In [8]:
# vamos criar uma pasta Tickers com o comando SO/Linux mkdir

!ls
!mkdir tickers

sample_data


In [9]:
#Para verificar se o Folder existe, senão, criamos...
from pathlib import Path
import os.path

p = Path('/content/Tickers')
if not p.exists():
  !mkdir tickers


mkdir: cannot create directory ‘tickers’: File exists


Esta é uma função (UDF) que vai gerar um arquivo CSV na pasta corrente
Ela será chamada dentro da função getData que criamos no passo anterior.

In [10]:
def SaveData(df, filename):
  #df.to_csv('/content/tickers/'+filename+'.csv')
  #df.to_csv('/content/tickers/acumulado.csv', mode='a', header=True)
  if (os.path.exists('/content/tickers/acumulado.csv')): 
    df.to_csv('/content/tickers/acumulado.csv', mode='a', header=False)
  else:
    df.to_csv('/content/tickers/acumulado.csv', mode='a', header=True)


Este loop irá vai criar um arquivo para cada Ticker da lista, com os dados do Dataframe

In [11]:
#for i in range(0,11):
#  df1= pd.read_csv('content/tickers/'+ str(files[i])+'.csv')
#  print (df1.head())

"""
Na versão original o script criava um arquivo para cada ticker,
o For abaixo chamará o a função getData com as alterações
já realizadas para criar e/ou armazenar em um só arquivo
acumulado.csv

Usando o ticker_list no for, deixamos o código mais organizado e 
inteligente, o numero de iterações é ajustado conforme 
a quantidade de parâmetros da lista, 
"""

for tik in ticker_list:
  print(tik)
  getData(tik)


SPY
SPY
[*********************100%***********************]  1 of 1 completed
SPY_2021-07-12 18:22:16
DOW
DOW
[*********************100%***********************]  1 of 1 completed
DOW_2021-07-12 18:22:16
LB
LB
[*********************100%***********************]  1 of 1 completed
LB_2021-07-12 18:22:16


In [56]:
#Criar uma coluna nova e calcular o indicador....


#novo dataframe
df_acum = pd.read_csv('/content/tickers/acumulado.csv')

df_acum.head()
print(df_acum.describe())

#Cálculo da média da coluna Volume
volmedia = df_acum['Volume'].mean()
print(volmedia)

#Criando nova coluna para receber o indicador
df_acummed = df_acum.assign(Indicador=0)

print(df_acummed.head())

#Populando a coluna nova com o valor do indicador
df_acummed.loc[df_acummed.Volume>=volmedia,'Indicador']=1
df_acummed.loc[df_acummed.Volume<volmedia,'Indicador']=0


print(df_acummed.head())


             Open        High         Low       Close   Adj Close        Volume
count    3.000000    3.000000    3.000000    3.000000    3.000000  3.000000e+00
mean   190.816664  192.736669  190.166668  192.333332  192.333332  1.003886e+07
std    211.924899  211.557057  212.084818  211.878250  211.878250  1.286247e+07
min     62.560001   63.459999   61.980000   63.450001   63.450001  1.936966e+06
25%     68.510000   70.665001   67.765001   70.065001   70.065001  2.623284e+06
50%     74.459999   77.870003   73.550003   76.680000   76.680000  3.309601e+06
75%    254.944996  257.375004  254.260002  256.774998  256.774998  1.408980e+07
max    435.429993  436.880005  434.970001  436.869995  436.869995  2.487000e+07
10038857.333333334
         Date        Open        High  ...    Volume  Ticker  Indicador
0  2021-07-12  435.429993  436.880005  ...  24870005     SPY          0
1  2021-07-12   62.560001   63.459999  ...   1936966     DOW          0
2  2021-07-12   74.459999   77.870003  ...   

In [28]:
df = pd.DataFrame({'c1': [10, 11, 12], 'c2': [100, 110, 120]})

for index, row in df.iterrows():
    print(row['c1'], row['c2'])

10 100
11 110
12 120


**Para saber mais:**

Sobre a biblioteca yfinance consulte https://pypi.org/project/yfinance/

sobre magic functions consulte: https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-lsmagic




In [3]:
import os.path
if(os.path.isfile('/content/sample_data/anscombe.json')):
  print("O arquivo existe")
else:
  print("O arquivo não existe")

O arquivo existe
